In [16]:
!pip install flask-ngrok

In [17]:
!pip install surprise

In [20]:
import pandas as pd
import pickle
import requests
import json
from flask import Flask, request, jsonify
import surprise
from flask_ngrok import run_with_ngrok
from flask import Flask
import re

In [21]:
def string_preprocess(row):
  text = row
  text = str(text)
  text = re.sub(r'\(\d+\)', '', text)
  text = text.lower()
  return text

In [23]:
app = Flask(__name__)
run_with_ngrok(app)
movie_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/movie.csv')
rating_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/rating.csv')

combined_small = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/combined_small.csv')
model = surprise.dump.load('/content/drive/MyDrive/Colab Notebooks/movie_lens/my_model_v2')
predictions = model[0]
print(predictions[0])
#create index item name and value item id series
movie_rating_combined = pd.merge(movie_df,rating_df,on='movieId')
movie_info_small = movie_rating_combined.iloc[:5000000]
movie_info_small['title'] = movie_info_small['title'].apply(string_preprocess)
movies_list_id = pd.Series(movie_info_small['movieId'].unique(),
                            index=movie_info_small['title'].unique())


@app.route('/api',methods=['GET'])
def get_api_recommendation():
  item_id = []
  movie_name = request.args.get('title')
  print('-------------------------->',movie_name)
  movie_id = movies_list_id.loc[movies_list_id.index == movie_name][0]
  user_id = combined_small.loc[combined_small.item == movie_id]['user'].tolist()[:5]
  print('useeeeeeeeeeeeer:',user_id)
  for id in user_id:
    for i in range(len(predictions)):
      if predictions[i].uid == id:
        print("--------------------> I am inside if",predictions[i])
        item_id.append(predictions[i].iid)
  movies_names = movies_list_id.loc[movies_list_id.isin(item_id)].index.tolist()[:5]
  return jsonify(movies_names)

@app.route('/movie',methods=['GET'])
def get_movies_recommendation():
  movie_name = request.args.get('title')
  movie_name = movie_name.lower()
  item_id = []
  rating = []
  movie_id = movies_list_id.loc[movies_list_id.index.str.contains(movie_name)][0]
  user_id = combined_small.loc[combined_small.item == movie_id]['user'].tolist()[:5]
  for id in user_id:
    for i in range(len(predictions)):
      if predictions[i].uid == id:
        item_id.append(predictions[i].iid)
        rating.append(predictions[i].est)
  movies_names = movies_list_id.loc[movies_list_id.isin(item_id)].index.tolist()[:5]
  for names in movies_names:
    if names.strip() == movie_name:
      movies_names.remove(names)
      movies_names.append(movies_list_id.loc[movies_list_id.isin(item_id)].index.tolist()[6])
  final = dict(zip(movies_names,rating))
  return jsonify(final)

if __name__=='__main__':
    app.run()
    

user: 26572      item: 36         r_ui = 4.00   est = 3.31   {'was_impossible': False}


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ec468098cb6a.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [26/Nov/2020 20:56:05] "GET /movie?title=goldeneye HTTP/1.1" 200 -
